<a href="https://colab.research.google.com/github/rafaelturon/pocs/blob/master/ml_modelo_regressao_carros_usados.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Machine Learning
###Exemplo de modelo de regressão p/ previsão de preços de venda de carros usados
* Este tutorial foi inspirado no repositório [Machine Learning](https://github.com/lfbraz/azure-databricks/blob/master/notebooks/ml-modelo-regressao-carros-usados.ipynb).

### Neste tutorial faremos uma [regressão linear simples](https://pt.wikipedia.org/wiki/Regress%C3%A3o_linear_simples) em que o objetivo será prever o PREÇO (y) de venda de um determinado modelo de veículo se baseando em algumas variáveis de entrada (X) 

A base de dados utilizada pode ser obtida no [link](https://github.com/lfbraz/machine-learning-tutorial/blob/master/datasets/dataset-carros-usados.csv) e foi baseada na versão [original](https://databricksdemostore.blob.core.windows.net/data/02.02/UsedCars.csv) disponibilizada pela Databricks, em que foi adaptada e traduzida para Português-Brasil.

##Importar base de dados
Utilizarei o [Google Colab](https://colab.research.google.com/) para treino do modelo, porém este tutorial pode ser utilizado com qualquer plataforma (sendo necessário apenas que o método de importação seja adaptado).

# **Running Pyspark in Colab**

Spark is a unified analytics engine for large-scale data processing. It provides high-level APIs in Scala, Java, Python, and R, and an optimized engine that supports general computation graphs for data analysis. It also supports a rich set of higher-level tools including Spark SQL for SQL and DataFrames, MLlib for machine learning, GraphX for graph processing, and Structured Streaming for stream processing.

https://spark.apache.org/

In [ ]:
!pip install pyspark

##Ler o arquivo .csv em um Spark DataFrame

Utilizaremos as segmentos opções: <br/>
<br/>
* format: CSV
* inferSchema: true / Permite que os tipos de dados sejam automaticamente inferidos
* header: true / Primeira linha será entendida como o cabeçalho
* sep: ";" / Será utilizado como delimitador de colunas o carácter ";"
* load: path / O caminho do arquivo que será carregado

In [ ]:
import pyspark
from pyspark import SparkFiles

filename = "dataset-carros-usados.csv"
url = "https://raw.githubusercontent.com/lfbraz/machine-learning-tutorial/master/datasets/{}".format(filename)

spark = SparkSession.builder.appName('pandasToSparkDF').getOrCreate()
spark.sparkContext.addFile(url)

file_type = "csv"
infer_schema = "true"
first_row_is_header = "true"
delimiter = ";"

carros_usados = spark.read.format(file_type) \
                     .option("inferSchema", infer_schema) \
                     .option("header", first_row_is_header) \
                     .option("sep", delimiter) \
                     .load("file://"+SparkFiles.get(filename))

## Visualização dos Dados

A tabela de carros usados possui os seguintes campos:<br/>
<br/>
* **PRECO**: Preço de venda do veículo (variável target / previsão)
* **IDADE_ANOS**: Número de anos desde a data de fabricação do veículo
* **KM**: Número de kilometros rodados pelo veículo
* **TIPO_COMBUSTIVEL**: Tipo de combustível utilizado
* **[HP](https://en.wikipedia.org/wiki/Horsepower)**: Medida de potência do veículo
* **COR_METALICA**: Indica se o veículo possui cor metálica (1 para sim e 0 para não). Pode indicar maior valorização
* **AUTOMATICO**: Indica se o veículo é automático (1 para sim e 0 para não)
* **[CC](https://pt.wikipedia.org/wiki/Cilindrada)**: Cilindradas do veículo
* **QTD_PORTAS**: Número de portas do veículo
* **PESO_KG**: Peso em quilograma do veículo

Com o comando `display(carros_usados)` podemos analisar o DataFrame

In [ ]:
carros_usados.show(n=5)

Executamos o Display do DataFrame podemos na própria célula indicar os tipos de gráficos que queremos visualizar (clicando no icone gráfico abaixo do DataFrame e em `PlotOptions`).

Abaixo é possível com o comando `Display` exibir uma visualização por Tipo de Combustível e Preço (utilizando 30 `bins` [número de colunas] ) alterando-se o `Plot Options` (somente disponível para Azure Databricks).

In [ ]:
display(carros_usados)

Também podemos utilizar outras bibliotecas para análise gráfica como o `matplotlib` ou `seaborn`. Neste caso pode-se converter o DataFrame Spark para um DataFrame Pandas (com isso a performance pode ser degradada).

Abaixo um [histograma](https://pt.wikipedia.org/wiki/Histograma) gerado a partir do preço de venda do veículo utilizando o matplotlib a partir de um [DataFrame do Pandas](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.html).

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline 

display(carros_usados.toPandas().plot(kind='hist', y='PRECO'))

# Criando uma regressão linear simples

Podemos iniciar investigando se a idade do veículo pode influenciar seu preço de venda.

In [ ]:
carros_usados_pd = carros_usados.toPandas()

fig, ax = plt.subplots()

# Populate the figure
plt.scatter(carros_usados_pd['IDADE_ANOS'], carros_usados_pd['PRECO'])

# Set various labels
plt.title('Preço dos carros usados como uma função da idade')
plt.ylabel('Preço [R$]')
plt.xlabel('Idade [Meses]')

# Extras?
plt.grid() # Turn plot-grid on

# Show figure
display(fig)

Repare que quanto maior a idade do veículo menor o seu preço de venda (make sense??).

Geralmente quando falamos em Machine Learning (aprendizado de máquina) estamos pensando em uma tarefa específica. No nosso caso, a tarefa básica 
é a de **prever o preço de venda de um veículo** utilizando para isso dados históricos de vendas de outros veículos.

Regressão Linear é uma das primeiras técnicas estatísticas aprendidas para previsão de dados que se comportam de forma linear 🙄. <br/><br/>
E o que isso significa ? Basicamente estamos falando que quando uma variável cresce (ou diminui) outra variável também têm o mesmo comportamento, como por exemplo, o **PREÇO** do veículo com a sua **IDADE** em meses (que acabamos de analisar de forma gráfica).

Este comportamento, para os mais entendidos, é feito através da famosa equação \\(y = ax + b\\).

No exemplo dado estamos falando apenas na relação de duas variáveis (PREÇO e IDADE), porém a ideia é utilizar diferentes variáveis no mesmo modelo linear.

Neste tipo de situação (que é a mais comum) a visualização é mais complexa pois a previsão não se dará por uma única reta (mas sim por hiperplanos 😳). Quem quiser entender mais sobre isso, recomendo o vídeo [Regressão Linear Múltipla](https://drive.google.com/file/d/1MKIO-oe8mtz92rlZp3eZJIYVmQSpW0Pi/view) de uma aula dada no IME/USP sobre o assunto.

## Tratamento dos dados
Antes de aplicarmos um modelo de Regressão Linear precisamos primeiramente tratar os dados que serão ENTRADA do modelo, isto é, precisaremos limpar, padronizar e enriquecer os dados que serão utilizados para treinamento do modelo.

In [ ]:
carros_usados.show(n=5)

O primeiro ponto importante é analisarmos se existem valores faltantes para cada uma das colunas que serão utilizadas no modelo. 

Repare que existem valores `null` (faltantes) no conjunto de dados analisado. Podemos checar estes valores através do comando abaixo:

In [ ]:
from pyspark.sql.functions import isnan, when, col

for c in carros_usados.columns:
  carros_usados.where(col(c).isNull()).show()

Existem diversas técnicas de tratamento de valores faltantes (substituição pela média, moda, etc). Para simplificarmos, o processo vamos simplesmente remover toda a linha em que seja encontrado algum valor faltante.

In [ ]:
carros_usados = carros_usados.na.drop()

Outro ponto importante é analisarmos o *TIPO* das variáveis de entrada. Repare que a variável **TIPO_COMBUSTIVEL** é uma variável categórica em formato TEXTO e quando falamos em modelos de aprendizado de máquina precisamos que todas as variáveis de *INPUT* sejam de alguma forma retratadas de forma numérica. Desta forma, o modelo (que nada mais é do que uma expressão matemática, muitas vezes extremamente complexa) conseguirá utilizar os dados de forma apropriada.

Para tratamento da variável utilizaremos as técnicas `StringIndexer` e `OneHotEncoder` que permitem representar as varíaveis categóricas em um formato vetorial binário. 

Com a `StringIndexer` representaremos as varíaveis categóricas de forma numérica (ex: GASOLINA=0, DIESEL=1, etc).

In [ ]:
from pyspark.ml.feature import StringIndexer

indexer = StringIndexer(inputCol='TIPO_COMBUSTIVEL', outputCol='TIPO_COMBUSTIVEL_index')
indexed = indexer.fit(carros_usados).transform(carros_usados)
indexed.show()

Repare que foi adicionada uma nova coluna chamada **TIPO_COMBUSTIVEL_index** com a representação numérica mencionada.

Também vamos utilizar a técnica de `OneHotEncoder` para transformação vetorial binária da variável que foi indexada (uma lista mais completa das técnicas pode ser encontrada no [link](https://spark.apache.org/docs/latest/ml-features.html) ). Para otimização deste processo ainda não vamos executar as transformações, vamos criar `Stages` (estágios de processamento das transformações) que serão posteriormente colocadas em um `Pipeline`).

In [ ]:
from pyspark.ml.feature import OneHotEncoder

encoder = OneHotEncoder(inputCols=["TIPO_COMBUSTIVEL_index"],
                                 outputCols=["TIPO_COMBUSTIVEL_vetor"])

stages = [indexer, encoder]

A variável `stages` contém os estágios de `StringIndexer` e `OneHotEncoder`, além deles utilizaremos também a técnica de `VectorAssembler` para consolidar todas as variáveis (features) que serão utilizadas para treinamento do modelo e adicionaremos mais este estágio na variável `stages`.

In [ ]:
from pyspark.ml.feature import VectorAssembler
colunas_treino = ['IDADE_ANOS', 'KM', 'HP', 'COR_METALICA', 'AUTOMATICO', 'CC', 'QTD_PORTAS', 'PESO_KG', 'TIPO_COMBUSTIVEL_vetor']

assembler = VectorAssembler(inputCols=colunas_treino, outputCol="features")
stages += [assembler]

Com os estágios de tratamento já definidos devemos agora dividir o conjunto de dados entre dados de treino (utilizado para treinar o modelo) e testes (utilizado para avaliar a performance do modelo). Utilizaremos a proporção de 80% para treino e 20% para teste.

In [ ]:
treino, teste = carros_usados.randomSplit([0.8, 0.2])

Com isso, finalmente 😆 podemos então realizar o treinamento de um modelo de Regressão Linear Simples.

Para isso vamos adicioná-lo no [`Pipeline`](https://spark.apache.org/docs/latest/api/python/pyspark.ml.html?highlight=onehotencoder#pyspark.ml.Pipeline) em conjunto com os outros estágios de tratamento de dados.

In [ ]:
from pyspark.ml.regression import LinearRegression
from pyspark.ml import Pipeline

lr = LinearRegression(featuresCol = 'features', labelCol='PRECO')
stages += [lr]
 
partialPipeline = Pipeline().setStages(stages)
model = partialPipeline.fit(treino)

##Persistir o modelo

Para ser possível a reutilização do modelo treinado podemos persisti-lo para que possa ser carregado posteriormente (sem a necessidade de re-treinamento).

Mount Google Drive to save sample levels as they are generated.

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
# Persist model
model_save_name = 'carros_usados_model'
path = F"/content/gdrive/My Drive/{model_save_name}" 
model.save(path)

In [ ]:
# Load model
from pyspark.ml import PipelineModel
model = PipelineModel.load(path)

Agora podemos também utilizar o modelo para fazer as predições na base de teste

## Predições na base TESTE

In [ ]:
predicoes = model.transform(teste)

predicoes.show(n=5)

## Podemos também analisar os resultados das predições

Com o modelo treinado e as predições realizadas, vamos analisar a métrica de Erro quadrático médio [RSME](https://en.wikipedia.org/wiki/Root-mean-square_deviation).

In [ ]:
from pyspark.ml.evaluation import RegressionEvaluator


evaluator = RegressionEvaluator(labelCol="PRECO", predictionCol="prediction", metricName="rmse")
rmse = evaluator.evaluate(predicoes)

print("RMSE na base TESTE = %g" % rmse)

### Podemos comparar graficamente o "ERRO" das predições realizadas com relação ao valor real de PREÇO.

In [ ]:
from sklearn import metrics

fig, ax = plt.subplots()

y_pred = predicoes.toPandas().prediction
y_test = predicoes.toPandas().PRECO

# Make a list of all the errors in the test-dataset:
errors = (y_pred - y_test)

### Populate the figure
# Plot the test-data:
plt.scatter(teste.toPandas().PRECO, errors, color='red', edgecolors='black')

# Set various labels
plt.ylabel('ERROS')
plt.xlabel('PREÇO')

plt.title('Erros em $ por Preço')

# Extras?
plt.grid() # Turn plot-grid on
plt.legend()

# Show figure
display(fig)